In [1]:
val sparkDummy = spark
import sparkDummy.implicits._

# Comparing Structured Streaming with DStream
<!-- We'll compare the Spark Structured Streaming and legacy DStream APIs to discuss their syntactic as well as semantic differences, and their relative strengths and weaknesses. -->

The old Spark streaming API used discretized streams or DStreams.  The model was that rather than a single, ever-growing dataset, a discretized stream provided an RDD at each trigger.  In a sense, it was a series of discretized batch operations.

![](images/streaming-flow.png)

<!--- Images from http://spark.apache.org/docs/latest/streaming-programming-guide.html --->

The model had several drawbacks:
1. DStream made it hard to deal with late or out-of-order data because the DStream was just discretized batches so it was difficult to update an old batch.
1. The API for DStream was very different from the API for RDDs because their underlying data model was different.
1. The streaming followed-by batch semantics made reliability hard.  If one step failed, the semantics of rerunning were unclear.

Nonetheless, it is still more built out than Structured Streaming and the API has feature support.  DStream will play a role in your streaming Spark applications for the foreseeable future.

# Custom Receivers in Spark DStream
<!-- DStreams give the ability to build your own custom streaming source.  We demonstrate how to build a custom receiver. -->

DStreams have support for custom receivers (i.e. sources).  For example, we are able to create a timed file source, which uses a separate thread to read data from a file and transmit the contents line by line, one each second.  The key is to extend the `Receiver` class and to use the `store` method to transmit.

In [3]:
import scala.io.Source
import org.apache.spark.streaming.receiver.Receiver
import org.apache.spark.storage.StorageLevel

// Should be a class ...
def timedFileSource(fileName: String) = {
    new Receiver[String](StorageLevel.MEMORY_AND_DISK_2) {
        def onStart() {
            new Thread("Timed File Source") {
                override def run() { receive() }
            }.start()
        }

        def onStop() { }

        private def receive() {
            for (line <- Source.fromFile(fileName).getLines) {
                println(line)  // print for debugging
                store(line)  // send the line as a source
                Thread.sleep(1000L)  // Wait one second
            }
        }
    }
}

# Iterative Wordcount Using Spark DStream
<!-- One of the tricky aspects of DStreams is that while their API appears similar to that of RDDs, their semantics are not necessarily the obvious parallel.  We'll illustrate a streaming RDD example. -->

To illustrate the difference in APIs, we will write wordcount using DStream.  As we'll see, the syntax appears similar but the semantics are vastly different.
![dstream-ops](images/streaming-dstream.png)

Because each discretized RDD is its own entity, we only see words in each time period and the wordcount will be per period, not the cumulative wordcount.
![dstream-ops](images/streaming-dstream-ops.png)

<!--- Images from http://spark.apache.org/docs/latest/streaming-programming-guide.html --->

In [4]:
import org.apache.spark._
import org.apache.spark.streaming._

val ssc = new StreamingContext(sc, Seconds(1))

In [5]:
(ssc.receiverStream(timedFileSource("data/summer.txt"))
    .flatMap(_.split(" "))
    .map(word => (word, 1))
    .reduceByKey(_ + _)
    .print())

ssc.start()             // Start the computation

# Cumulative Wordcount using Spark DStream
<!-- Keeping track of state between batch times for cumulative statistics is one of the more complex aspects of DStream.  We'll explain how and demonstrate it in a working sample.  -->

The above only gives the iterative (non-cumulative) wordcount, so how do we get the cumulative wordcount?  It turns out we need to keep track of state using separate `State` variables and manually update it in calls to `mapWithState`.  We will create a `StateSpec` object, which is a thin wrapper around our `updateFn` function.  This is called by `mapWithState` for key-value pairs.  It must perform two functions:

1. Update the state variable that's passed in (side effects!)
2. Return a resulting RDD

The API is definitely more involved and more cumbersome.

In [5]:
// You cannot create jobs in a started streaming context
// Consider restarting the kernel

def updateFn(key: String, value: Option[Int], state: State[Int]) = {
    // update state (stateful!)
    state.update(value.getOrElse(0) + state.getOption.getOrElse(0))

    // result to return
    (key, state.get)
}

val spec = StateSpec.function(updateFn _)

// checkpointing is mandatory
ssc.checkpoint("_checkpoints")

(ssc.receiverStream(timedFileSource("data/summer.txt"))
    .flatMap(_.split(" "))
    .map(word => (word, 1))
    .mapWithState(spec)
    .print())

ssc.start()             // Start the computation

# Benefits of Spark Tungsten
<!-- One of the benefits of Structured Streaming is that it takes advantage of recent low-level Spark optimizations.  We'll explain what these are in this segment. -->

We've seen the surface-level API differences between DStream and Structured Streaming. There are also many differences under the hood.  These have been added as newer versions of Spark for Datasets and Structured Streaming and are sometimes retrofitted to RDDs.

Spark has traditionally been focused on optimizing for inter-computer network IO Efficiency.  This was the major bottleneck in MapReduce and other traditional distributed computing systems.  At this point, Spark is now intra-computer memory- and CPU-bound.

1. **Customized Memory Management:** Spark understands its own memory allocation needs better than the generic JVM garbage collector.  Tungsten takes advantage of this by using `sun.misc.Unsafe`, which exposes C-Style off-heap memory access.  The result is fewer unnecessary garbage collection events and improved performance.
1. **Binary Encoding:** While Spark traditionally needed to serialize data to JVM objects, it now uses the Tungsten binary encoding.  This has two major advantages.  First, it decreases the memory footprint.  While "ABCD" would take 4 bytes of UTF-8 to encode, it would be stored using 48 bytes as a JVM object.  Second, instead of serializing into objects, it's able to perform many actions directly on the the raw binary encoding, reducing the computational overhead for serialization / deserialization.
1. **Code Generation:** Spark historically used generic JVM function evaluation.  Given the virtual function lookups, automated boxing of primitive types, and other JVM overhead, this dramatically slows down the computation.  By using type information, Tungsten is able to generate byte code and speed up performance.
1. **Cache-aware Computation:** Tungsten lays out its memory in a way that takes advantage of CPU cache locality to reduce cache spilling and speed up computations.

For more information, check out [this blog post](https://databricks.com/blog/2015/04/28/project-tungsten-bringing-spark-closer-to-bare-metal.html) or [this presentation](http://www.slideshare.net/SparkSummit/deep-dive-into-project-tungsten-josh-rosen).

# Tungsten Performance Benefit Demonstration
<!-- In this segment, we'll demonstrate how to use Spark's UI to view the memory usage of Spark and demonstrate how Spark Tungsten can reduce memory usage. -->

The following two operations count a million integers in memory.
- The first uses RDDs and must serialize all the objects into Java Objects.
- The second use sTungsten and uses a more compact binary encoding.

Tungsten saves a factor of nearly 4 on memory:

![Tungsten Memory](images/Tungsten_Memory.png)

To reproduce the example, run the following code and go to your Spark UI Viewer.  This should be at [http://localhost:4040/storage/](http://localhost:4040/storage/) but may be at a different port if you have started multiple Spark kernels.

In [3]:
val million = sc.parallelize(0 until math.pow(10, 6).toInt)

// Using RDDs
million.cache.count

// Using Tungsten
million.toDS.cache.count

# Benefits of Spark Catalyst
<!-- In addition to the previous low-level optimizations of Tungsten, Structured Streaming also takes advantage of the Catalyst, which optimizes Spark jobs at the query level.  We'll explain what these are in this segment. -->

In addition to the previous low-level optimizations of Tungsten, Structured Streaming also takes advantage of the Catalyst, which optimizes Spark jobs at the query level.  Optimizations include:
1. **Constant Folding:** Evaluating constant expressions at compile time rather than at run time.
1. **Predicate Pushdown:** Running operations that reduce the dataload (e.g. selecting columns or filtering rows) earlier in the query.  This reduces the amount of data that needs to be processed.
1. **Projection Pruning:** Only reading the columns (fields) used in the query from the database.
1. **Pipelining Operations:** Combines multiple projection and filter operations into a single map operation.
1. **Cost-based Optimization:** Spark actually builds multiple plans to compute a query, computes their cost, and chooses the cheapest one.
1. **Code Generation:** The final operational plan is transformed into optimized Java bytecode.

# Viewing Query Plans in Spark Shell
<!-- In this segment, we'll show how to debug query plans for the legacy RDDs and DStream APIs. -->

For an RDD, we can view the query plan calling the `.toDebugString` method.  Notice that multiple (even consecutive!) maps and filters are run as separate map and filter steps.

In [5]:
(million.map(_ + 1)
    .map(_ + 1)
    .filter(_ > 1)
    .filter(_ > 2 * 4)
    .toDebugString)

# Visualizing Query Stages in Spark UI Viewer
<!-- In this segment, we'll demonstrate how to use the Spark UI Viewer to view the individual stages of a Spark job. -->

You can also check out the "Stages" tab in the Spark UI viewer [http://localhost:4040/stages/](http://localhost:4040/stages/)  to check out the individual stages of a Spark job.

<img src="images/spark-stages.png" width="40%"/>

# Viewing Spark Catalyst-Optimzied Physical Plans
<!-- In this segment, we'll show how to debug query plans for the new Dataset and Structured Streaming APIs and show how Spark can optimize some fairly complex queries. -->

For a dataset, we can view the query plan with the `.explain` method.  You can see that it has performed several optimizations:
1. It grouped multiple (even non-consecutive!) `select` and `filter` operations together.
1. It pushed the `.filter` earlier in the query.
1. It performs constant folding by multiplying out `2 * 4`.

In [6]:
(million.toDS
    .select('value, 'value + 1 as 'value2)
    .select('value2 + 1 as 'value2)
    .filter('value >= 1)
    .select('value2 + 1 as 'value2)
    .filter('value >= 2 * 4)
    .explain)

<img src="images/logo-text.jpg" width="20%"/>